# 25. 데이터 전처리 및 피처 엔지니어링

## 25.1 사용 목적에 따라 모델 서식 지정하기
+ 데이터를 Double Type의 컬럼으로 가져와서 레이블을 표시, Vector 타입의 컬럼을 사용해서 특징을 나타냄
+ 그래프 분석은 정점과 에지가 각각 DataFrame으로 구성되어야 함

In [1]:
# 세션 생성
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("feature engineering example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark.conf.set('spark.sql.shuffle.partitions', 5)

In [2]:
sales = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("../BookSamples/data/retail-data/by-day/*.csv")\
    .coalesce(5)\
    .where("Description IS NOT NULL")
    # 널 값을 꼭 걸러내야 함

In [3]:
fakeIntDF = spark.read.parquet("../BookSamples/data/simple-ml-integers/")
simpleDF = spark.read.json("../BookSamples/data/simple-ml/")
scaleDF = spark.read.parquet("../BookSamples/data/simple-ml-scaling/")

In [4]:
sales.cache()
sales.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [5]:
fakeIntDF.show()
simpleDF.show()
scaleDF.show()

+----+----+----+
|int1|int2|int3|
+----+----+----+
|   7|   8|   9|
|   1|   2|   3|
|   4|   5|   6|
+----+----+----+

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green|good|    15| 38.97187133755819|
|green|good|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     1| 38.97187133755819|
|  red| bad|     2|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|  red|good|    45| 38.97187133755819|
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green|good|    15| 38.97187133755819|
|green|good|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     1| 38.97187133755819|
|  red| bad|     2|14.

## 25.2 변환자
+ 변환자는 다양한 방식으로 원시 데이터를 변환시키는 함수
+ 새로운 상호작용 변수를 생성하거나 컬럼을 정규화, 모델 입력을 위해 Double형 변환 기능 등을 제공

In [6]:
from pyspark.ml.feature import Tokenizer

tkn = Tokenizer(inputCol="Description", outputCol="words")
tkn.transform(sales.select("Description")).show()

+--------------------+--------------------+
|         Description|               words|
+--------------------+--------------------+
|  RABBIT NIGHT LIGHT|[rabbit, night, l...|
| DOUGHNUT LIP GLOSS |[doughnut, lip, g...|
|12 MESSAGE CARDS ...|[12, message, car...|
|BLUE HARMONICA IN...|[blue, harmonica,...|
|   GUMBALL COAT RACK|[gumball, coat, r...|
|SKULLS  WATER TRA...|[skulls, , water,...|
|FELTCRAFT GIRL AM...|[feltcraft, girl,...|
|CAMOUFLAGE LED TORCH|[camouflage, led,...|
|WHITE SKULL HOT W...|[white, skull, ho...|
|ENGLISH ROSE HOT ...|[english, rose, h...|
|HOT WATER BOTTLE ...|[hot, water, bott...|
|SCOTTIE DOG HOT W...|[scottie, dog, ho...|
|ROSE CARAVAN DOOR...|[rose, caravan, d...|
|GINGHAM HEART  DO...|[gingham, heart, ...|
|STORAGE TIN VINTA...|[storage, tin, vi...|
|SET OF 4 KNICK KN...|[set, of, 4, knic...|
|      POPCORN HOLDER|   [popcorn, holder]|
|GROW A FLYTRAP OR...|[grow, a, flytrap...|
|AIRLINE BAG VINTA...|[airline, bag, vi...|
|AIRLINE BAG VINTA...|[airline, 

## 25.3 전처리 추정자
+ 수행하려는 변환이 입력 컬럼에 대한 데이터 또는 정보로 초기화되어야 할 때 사용
    + StandardScaler는 평균과 분산 파라미터가 필요하므로 이를 추정자가 처리함
+ inputCol, outputCol 지정 필요

In [7]:
from pyspark.ml.feature import StandardScaler

ss = StandardScaler(inputCol="features", outputCol="scaledFeatures")
ss.fit(scaleDF).transform(scaleDF).show()

+---+--------------+--------------------+
| id|      features|      scaledFeatures|
+---+--------------+--------------------+
|  0|[1.0,0.1,-1.0]|[1.19522860933439...|
|  1| [2.0,1.1,1.0]|[2.39045721866878...|
|  0|[1.0,0.1,-1.0]|[1.19522860933439...|
|  1| [2.0,1.1,1.0]|[2.39045721866878...|
|  1|[3.0,10.1,3.0]|[3.58568582800318...|
+---+--------------+--------------------+



## 25.4 고수준 변환자
+ 비즈니스 문제에 집중하기 위해서는 가능한 한 최상위 수준의 변환자 사용을 지향해야 함

### 25.4.1 RFormula
+ R에서 빌려옴
+ 데이터값은 숫자형 또는 범주형이 되고, 문자열에서 값을 추출하는 등의 조직을 할 필요가 없음
+ 기본 연산자
    + ~
        + 함수에서 타깃과 항을 분리
    + \+
        + +0는 y 절편 제거를 의미
    + -
        + 삭제기호, -1는 y 절편 제거를 의미 +0과 결과가 같음
    + :
        + 상호작용(수치형 값이나 이진화된 범주 값에 대한 곱셈)
    + .
        + 타깃 / 종속변수를 제외한 모든 컬럼
        